In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# Import plotting library
import matplotlib.pyplot as plt

# Set global plot background color
plt.rcParams["figure.facecolor"] = "w"

# Import for SVM classifier
from sklearn.svm import SVC

# Imports for dataset and model selection
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

# Imports for QSVC classifier
from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.kernels import QuantumKernel

# Set the random seed for QSVC
seed = 12345
algorithm_globals.random_seed = seed

In [25]:
features = np.random.rand(100, 3)
label = np.array([np.round(sum(f)/3.0 + np.random.normal(0, .25)) for f in features])

print(label)

[ 0.  0.  1.  1.  0.  0.  0.  0.  1.  1.  1.  0.  0.  1. -0.  1.  0.  0.
  0.  0.  1.  1.  0.  0.  0.  1. -0.  0.  0.  1.  1.  1.  0.  1.  1.  0.
  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  1.  1.  0.  1.  0.
  0.  1.  1.  1.  1.  1.  0.  1.  0.  0.  1.  1.  1.  0.  0.  0.  1.  0.
  1. -0.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  1.  1.  0.  1.  1.  0.
  1.  0.  0.  0. -0.  1.  1.  1.  0.  1.]


In [52]:
labels = np.array([0 if (sum(f)/3.0 + np.random.normal(0, .2)) < 0.5 else 1 for f in features])
labels

array([1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0])

In [81]:
def get_data():    
    features = np.random.rand(100, 3)
    labels = np.array([0 if (sum(f)/3.0 + np.random.normal(0, .1)) < 0.5 else 1 for f in features])
    return features, labels


def split_train_test_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
    return X_train, X_test, y_train, y_test


def train_qsvc(X_train, y_train):
    feature_map = ZZFeatureMap(feature_dimension=3, reps=4, entanglement="linear")
    backend = QuantumInstance(
        BasicAer.get_backend("qasm_simulator"), shots=16, seed_simulator=seed, seed_transpiler=seed
    )
    kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)
    qsvc = SVC(kernel=kernel.evaluate)
    qsvc.fit(X_train, y_train)
    import pickle

    # save the model to disk
    filename = 'finalized_model.sav'
    pickle.dump(qsvc_model, open(filename, 'wb'))
    return qsvc


def predict_example(example):
    import pickle
    filename = 'finalized_model.sav'
    trained_model = pickle.load(open(filename, 'rb'))
    feature_map = ZZFeatureMap(feature_dimension=3, reps=4, entanglement="linear")
    backend = QuantumInstance(
        BasicAer.get_backend("qasm_simulator"), shots=16, seed_simulator=seed, seed_transpiler=seed
    )
    kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)
    qsvc = SVC(kernel=kernel.evaluate)
    qsvc.set_params(trained_model.load_params())
    return loaded_model.predict(example)

In [82]:
def train_workflow():
    X, y = get_data()
    X_train, X_test, y_train, y_test = split_train_test_data(X=X, y=y)
    qsvc_model = train_qsvc(X_train=X_train, y_train=y_train)
    return X_test, y_test, qsvc_model
    

In [83]:
X_test, y_test, qsvc_model = workflow()
y_pred = qsvc_model.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.75      0.69        12
           1       0.73      0.62      0.67        13

    accuracy                           0.68        25
   macro avg       0.69      0.68      0.68        25
weighted avg       0.69      0.68      0.68        25



In [50]:
import pickle

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(qsvc_model, open(filename, 'wb'))

In [94]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8


In [95]:
np.random.normal(0, .1)

-0.1591859939469502

In [97]:
print(loaded_model.get_params())

{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': <bound method QuantumKernel.evaluate of <qiskit_machine_learning.kernels.quantum_kernel.QuantumKernel object at 0x7f97d267cfa0>>, 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [100]:
filename = 'finalized_model.sav'
trained_model = pickle.load(open(filename, 'rb'))
feature_map = ZZFeatureMap(feature_dimension=3, reps=4, entanglement="linear")
backend = QuantumInstance(
    BasicAer.get_backend("qasm_simulator"), shots=16, seed_simulator=seed, seed_transpiler=seed
)
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)
qsvc = SVC(kernel=kernel.evaluate)
#print(qsvc.score(X_test, y_test))
print(trained_model.get_params())
qsvc.set_params(trained_model.get_params())
print(qsvc.score(X_test, y_test))


{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': <bound method QuantumKernel.evaluate of <qiskit_machine_learning.kernels.quantum_kernel.QuantumKernel object at 0x7f97d535b2b0>>, 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Traceback (most recent call last):
  Input In [100] in <cell line: 11>
    qsvc.set_params(trained_model.get_params())
TypeError: set_params() takes 1 positional argument but 2 were given

Use %tb to get the full traceback.
